### Loading required libraries

In [1]:
import vanna
import os
from dotenv import load_dotenv
from vanna.remote import VannaDefault
import sqlite3

### Setting up Vanna

In [2]:
# Connecting to vanna, we are using the defaul LLM (GPT 3.5) and Vector store (pgvector) provided by vanna

load_dotenv()
vanna_api_key = os.getenv("vanna_api_key")
vanna_model_name = "vannagetsomecoffee"
vn = VannaDefault(model=vanna_model_name, api_key=vanna_api_key)

### Setting up a template sqlite db

In [3]:
# Using a template db called 'chinook' downloaded from the web
db_conn = sqlite3.connect("chinook.db")

db_cursor = db_conn.cursor()

In [8]:
# This is the query we will use to fetch ddl information to feed to vanna 
result = db_cursor.execute("SELECT type, sql FROM sqlite_master WHERE sql is not null")

# To see that the query fetches uncomment this line below
#result.fetchall()

### Adding the DB and DB's ddl information to vanna 

The training is only needed once if you are using vanna's default vector DB hence I have commented out the training code

In [3]:
vn.connect_to_sqlite('chinook.db')

In [6]:
# df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

# for ddl in df_ddl['sql'].to_list():
#   vn.train(ddl=ddl)

Adding ddl: CREATE TABLE "albums"
(
    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)
Adding ddl: CREATE TABLE sqlite_sequence(name,seq)
Adding ddl: CREATE TABLE "artists"
(
    [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Name] NVARCHAR(120)
)
Adding ddl: CREATE TABLE "customers"
(
    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) 
		ON DELETE NO ACTION 

In [ ]:
vn.get_training_data()

### Querying Vanna

In [7]:
# Don't use vn.ask it calls several functions : generate_sql, run_sql, and two functions for creating plots
#vn.ask(question="How many customers are from brazil?")
query = vn.generate_sql("How many customers are from brazil?")

SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "customers"\r\n(\r\n    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [FirstName] NVARCHAR(40)  NOT NULL,\r\n    [LastName] NVARCHAR(20)  NOT NULL,\r\n    [Company] NVARCHAR(80),\r\n    [Address] NVARCHAR(70),\r\n    [City] NVARCHAR(40),\r\n    [State] NVARCHAR(40),\r\n    [Country] NVARCHAR(40),\r\n    [PostalCode] NVARCHAR(10),\r\n    [Phone] NVARCHAR(24),\r\n    [Fax] NVARCHAR(24),\r\n    [Email] NVARCHAR(60)  NOT NULL,\r\n    [SupportRepId] INTEGER,\r\n    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\nCREATE TABLE "invoices"\r\n(\r\n    [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [CustomerId] INTEGER  N

In [8]:
vn.run_sql(query)

,TotalCustomersFromBrazil
0,5


### Custom function for querying with Vanna

In [15]:
def askVanna(question):
    query = vn.generate_sql(question)
    print(vn.run_sql(query))

In [16]:
askVanna("What are the top 10 albums by sale and who are the artits of those albums")

SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "albums"\r\n(\r\n    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Title] NVARCHAR(160)  NOT NULL,\r\n    [ArtistId] INTEGER  NOT NULL,\r\n    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\nCREATE TABLE "artists"\r\n(\r\n    [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(120)\r\n)\n\nCREATE TABLE "tracks"\r\n(\r\n    [TrackId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(200)  NOT NULL,\r\n    [AlbumId] INTEGER,\r\n    [MediaTypeId] INTEGER  NOT NULL,\r\n    [GenreId] INTEGER,\r\n    [Composer] NVARCHAR(220),\r\n    [Milliseconds] INTEGER  NOT NULL,\r\n    [Bytes] INTEGER,\r\n    [Uni

In [4]:
# Correct way of using vanna.ask()

vn.ask(question="Give me a list of all countries that the customers are from", print_results=True, auto_train=False, visualize=False) 

SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "customers"\r\n(\r\n    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [FirstName] NVARCHAR(40)  NOT NULL,\r\n    [LastName] NVARCHAR(20)  NOT NULL,\r\n    [Company] NVARCHAR(80),\r\n    [Address] NVARCHAR(70),\r\n    [City] NVARCHAR(40),\r\n    [State] NVARCHAR(40),\r\n    [Country] NVARCHAR(40),\r\n    [PostalCode] NVARCHAR(10),\r\n    [Phone] NVARCHAR(24),\r\n    [Fax] NVARCHAR(24),\r\n    [Email] NVARCHAR(60)  NOT NULL,\r\n    [SupportRepId] INTEGER,\r\n    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\nCREATE TABLE "invoices"\r\n(\r\n    [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [CustomerId] INTEGER  N

('SELECT DISTINCT Country\nFROM customers;',
            Country
 0           Brazil
 1          Germany
 2           Canada
 3           Norway
 4   Czech Republic
 5          Austria
 6          Belgium
 7          Denmark
 8              USA
 9         Portugal
 10          France
 11         Finland
 12         Hungary
 13         Ireland
 14           Italy
 15     Netherlands
 16          Poland
 17           Spain
 18          Sweden
 19  United Kingdom
 20       Australia
 21       Argentina
 22           Chile
 23           India,
 None)

In [5]:
# Correct way of using vanna.ask()

vn.ask(question="Which tracks were purchased in the order with maximum tracks purchased?", print_results=True, auto_train=False, visualize=False) 

SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "tracks"\r\n(\r\n    [TrackId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(200)  NOT NULL,\r\n    [AlbumId] INTEGER,\r\n    [MediaTypeId] INTEGER  NOT NULL,\r\n    [GenreId] INTEGER,\r\n    [Composer] NVARCHAR(220),\r\n    [Milliseconds] INTEGER  NOT NULL,\r\n    [Bytes] INTEGER,\r\n    [UnitPrice] NUMERIC(10,2)  NOT NULL,\r\n    FOREIGN KEY ([AlbumId]) REFERENCES "albums" ([AlbumId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\r\n    FOREIGN KEY ([GenreId]) REFERENCES "genres" ([GenreId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\r\n    FOREIGN KEY ([MediaTypeId]) REFERENCES "media_types" ([MediaTypeId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\nCREATE INDEX [IFK_TrackA

('WITH max_order AS (\n    SELECT InvoiceId, COUNT(TrackId) AS tracks_purchased\n    FROM invoice_items\n    GROUP BY InvoiceId\n    ORDER BY tracks_purchased DESC\n    LIMIT 1\n)\n\nSELECT t.Name\nFROM tracks t\nJOIN invoice_items ii ON t.TrackId = ii.TrackId\nJOIN max_order mo ON ii.InvoiceId = mo.InvoiceId;',
                       Name
 0       Your Time Has Come
 1                Dandelion
 2      Rock 'N' Roll Music
 3               Moon germs
 4         Super Terrorizer
 5            Heart Of Gold
 6               Evil Woman
 7               Cornucopia
 8      Bowels Of The Devil
 9        Body Count Anthem
 10               Jerusalem
 11  When My Left Eye Jumps
 12               Meditação
 13               Esse Cara,
 None)

In [5]:
# Correct way of using vanna.ask()

vn.ask(question="Give me a list of all countries that the customers are from", print_results=True, auto_train=False, visualize=False) 

HTTPSConnectionPool(host='ask.vanna.ai', port=443): Max retries exceeded with url: /rpc (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))


(None, None, None)

In [5]:
vn.ask(question="Give me a list of all countries that the customers are from", print_results=True, auto_train=False, visualize=False) 


HTTPSConnectionPool(host='ask.vanna.ai', port=443): Max retries exceeded with url: /rpc (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))


(None, None, None)